In [1]:
import math
import torch

# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64   # use double precision

from torch.optim import Adam

from botorch.models import SingleTaskGP
from botorch.acquisition import ExpectedImprovement
from botorch.test_functions.synthetic import Labs
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.constraints import GreaterThan


In [2]:
# LABS benchmark with 30 binary variables
dim = 30
labs = Labs(dim=dim)


In [3]:
# Generate initial random binary training data
def generate_initial_data(n=8):
    X = torch.randint(0, 2, (n, dim), dtype=dtype, device=device)
    Y = labs(X).unsqueeze(-1)  # LABS merit factor values
    return X, Y

train_X, train_Y = generate_initial_data()
print("Initial best merit factor:", train_Y.max().item())


Initial best merit factor: 1.3595166163141994


In [4]:
model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))


In [5]:
mll = ExactMarginalLogLikelihood(model.likelihood, model).to(train_X)


In [6]:
optimizer = Adam(model.parameters(), lr=0.05)


In [7]:
NUM_EPOCHS = 150

model.train()
for epoch in range(NUM_EPOCHS):
    optimizer.zero_grad()
    output = model(train_X)
    loss = -mll(output, model.train_targets)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.3f}")


Epoch 10/150 - Loss: 13.684
Epoch 20/150 - Loss: 13.453
Epoch 30/150 - Loss: 13.243
Epoch 40/150 - Loss: 13.061
Epoch 50/150 - Loss: 12.909
Epoch 60/150 - Loss: 12.787
Epoch 70/150 - Loss: 12.693
Epoch 80/150 - Loss: 12.623
Epoch 90/150 - Loss: 12.573
Epoch 100/150 - Loss: 12.537
Epoch 110/150 - Loss: 12.514
Epoch 120/150 - Loss: 12.498
Epoch 130/150 - Loss: 12.488
Epoch 140/150 - Loss: 12.482
Epoch 150/150 - Loss: 12.479


In [8]:
N_BATCH = 5   # how many BO steps to run

for i in range(N_BATCH):
    # Re-fit GP on current data
    model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
    model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))
    mll = ExactMarginalLogLikelihood(model.likelihood, model).to(train_X)
    optimizer = Adam(model.parameters(), lr=0.05)

    # Train GP hyperparameters
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(train_X)
        loss = -mll(output, model.train_targets)
        loss.backward()
        optimizer.step()

    model.eval()

    # Acquisition function (Expected Improvement)
    EI = ExpectedImprovement(model=model, best_f=train_Y.max())

    # Sample candidate binary points
    candidates = torch.randint(0, 2, (2000, dim), dtype=dtype, device=device)
    acq_values = EI(candidates.unsqueeze(1))
    new_x = candidates[acq_values.argmax()]

    # Evaluate and add to training data
    new_y = labs(new_x.unsqueeze(0)).unsqueeze(-1)
    train_X = torch.cat([train_X, new_x.unsqueeze(0)], dim=0)
    train_Y = torch.cat([train_Y, new_y], dim=0)

    print(f"Iteration {i+1} | Best merit factor so far: {train_Y.max().item():.4f}")


/workspaces/botorch-demo/.venv/lib/python3.12/site-packages/botorch/acquisition/analytic.py:331: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


Iteration 1 | Best merit factor so far: 1.3595
Iteration 2 | Best merit factor so far: 1.4658
Iteration 3 | Best merit factor so far: 1.4658
Iteration 4 | Best merit factor so far: 2.9032
Iteration 5 | Best merit factor so far: 2.9032
